In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import warnings
import datetime
import time
import threading
import concurrent.futures
import multiprocessing
import requests

In [3]:
from datetime import timedelta
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint
#from sklearn.cluster import DBSCAN
#from sklearn.cluster import KMeans
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from itertools import combinations
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from multiprocessing import Pool, Process, Queue
from pyswarm import pso
#from pmdarima import auto_arima
#from statsmodels.tsa.arima.model import ARIMA
#from googletrans import Translator

In [4]:
def get_sp500_tickers():
    # Ссылка на страницу Wikipedia с перечнем компаний S&P 500
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

    # Скачивание таблицы с помощью pandas
    sp500_table = pd.read_html(url, header=0)[0]
    tickers = sp500_table['Symbol'].tolist()

    # Обработка символов, которые могут вызвать проблемы (например, замена '.' на '-')
    tickers = [ticker.replace('.', '-') for ticker in tickers]

    return tickers

# Получение списка тикеров
tickers = get_sp500_tickers()

start_date = "2022-01-02"
end_date = "2024-01-26"
data = yf.download(tickers, start=start_date, end=end_date)['Close']
data_open = yf.download(tickers, start=start_date, end=end_date)['Open']

[*********************100%%**********************]  503 of 503 completed
[*********************100%%**********************]  503 of 503 completed


In [5]:
data_2022 = yf.download(tickers, start="2022-01-02", end="2023-01-02")['Close']
data_2022_open = yf.download(tickers, start="2022-01-02", end="2023-01-02")['Open']
data_2023 = yf.download(tickers, start="2023-01-02", end="2024-01-02")['Close']
data_2023_open = yf.download(tickers, start="2023-01-02", end="2024-01-02")['Open']

[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['VLTO', 'KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 1641099600, endDate = 1672635600")
[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['VLTO', 'KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 1641099600, endDate = 1672635600")
[*********************100%%**********************]  503 of 503 completed
[*********************100%%**********************]  503 of 503 completed


In [514]:
# Удаляем столбцы, где более 10% данных отсутствуют
threshold = 0.1  # 10% порог
data = data.dropna(thresh=int((1 - threshold) * len(data)), axis=1)

# Заполнение оставшихся пропусков, например, методом ffill (forward fill)
data = data.ffill()

# Удаляем строки, которые все еще содержат NaN (если они есть)
data = data.dropna()

In [521]:
unique_tickers = list(data.columns)
num_tickers = len(unique_tickers)
pairs = list(combinations(data.columns, 2))
pairs[:5]

[('A', 'AAL'), ('A', 'AAPL'), ('A', 'ABBV'), ('A', 'ABNB'), ('A', 'ABT')]

In [96]:
pairs_filter = []
test = pairs[:1000]
# Фильтр пар
for i in pairs:
    tik1, tik2 = i
    adf = 0
    if tik1 in data.columns and tik2 in data.columns:
        stock1 = data[tik1]
        stock2 = data[tik2]
        # Рассчитываем коинтеграцию с помощью CADF теста
        score, p_value, _ = ts.coint(stock1, stock2)
        # ADF тест
        spread = stock1 - stock2
        adf_result = ts.adfuller(spread)
        # Корреляция Спирмена
        corr, spear_p = spearmanr(stock1, stock2)
        if adf_result[0] < adf_result[4]['1%']:
            adf = 1
        # Сохраняем результаты
        pairs_filter.append({'pair' : i, 'ADF' : adf, 'ADF-stat' : adf_result[0], 'CADF-score' : score, 'Spearman' : corr})
        adf = 0
pairs_filter = pd.DataFrame(pairs_filter)
pairs_filter.head()

,pair,ADF,ADF-stat,CADF-score,Spearman
0,"(A, AAL)",0,-2.271479,-2.244466,0.210703
1,"(A, AAPL)",0,-0.911358,-2.706030,-0.371842
2,"(A, ABBV)",0,-2.917184,-2.439999,0.388525
3,"(A, ABNB)",0,-1.577386,-2.346131,-0.153614
4,"(A, ABT)",0,-1.752335,-1.752627,0.360048


In [97]:
pairs_filter.to_csv('pairs_filter.csv')

In [504]:
filtred_pairs = []
for i in range(len(pairs_filter)):
    if pairs_filter.at[i, 'ADF'] == 1:
       filtred_pairs.append(pairs_filter.at[i, 'pair'])
    if pairs_filter.at[i, 'Spearman'] >= 0.7 and pairs_filter.at[i, 'CADF-score'] < -2.5:
        filtred_pairs.append(pairs_filter.at[i, 'pair'])
len(filtred_pairs)

11875

In [405]:
line_reg = data.copy()
deviation = data.copy()
relative_deviation = data.copy()
spread_pairs = data.copy()
mean_price = data.copy()
slope = data.copy()
slope.loc[:, :] = np.nan
mean_price.loc[:, :] = np.nan
spread_pairs.loc[:, :] = np.nan
line_reg.loc[:, :] = np.nan
deviation.loc[:, :] = np.nan
relative_deviation.loc[:, :] = np.nan

In [524]:
data_open = yf.download(unique_tickers, start=start_date, end=end_date)['Open']

[*********************100%%**********************]  500 of 500 completed


In [516]:
def regression(tickers, roll_window):

    '''Расчет линейной регрессии для каждого тикера'''

    # Перевод дат в цифровой формат
    line_reg['DateNumeric'] = data.index.map(pd.Timestamp.toordinal)

    # Создание индексов в качестве признаков
    X = line_reg['DateNumeric'].values.reshape(-1, 1)
    for tiker in unique_tickers:
        mean_price[tiker] = data[tiker].rolling(window=10).mean()
        for index in range(roll_window, len(data)):
            current_date = data.index[index]
            tiker_price = data.at[current_date, tiker]
            # Расчет отклонения от линии регрессии котировок
            numeric_date = current_date.toordinal()
            # Рассчет отклонения от линейной регрессии для каждого тикера
            model = LinearRegression()
            model.fit(X[index-roll_window:index], data[tiker][index-roll_window:index])
            line_reg.at[current_date, tiker] = model.predict([[numeric_date]])
            slope.at[current_date, tiker] = model.coef_[0]
            deviation.at[current_date, tiker] = tiker_price - line_reg.at[current_date, tiker]
            relative_deviation.at[current_date, tiker] = (tiker_price - line_reg.at[current_date, tiker]) / line_reg.at[current_date, tiker]

regression(unique_tickers, 23)

In [6]:
def pairs_trading(pair, options): #window, coef_entry, coef_sell, coef_loss, coef_profit):
    # Тестируемые тикеры акций
    name_1, name_2 = pair

    ticker1 = data[name_1]
    ticker2 = data[name_2]

    # Начальные параметры
    initial_capital = 100000    # Стартовый капитал
    max_cash = initial_capital    # Максимально достигнутая сумма
    cash = initial_capital    # Баланс кошелька
    low_shares = 0    # Акции low
    jbht_shares = 0    # Акции jbht
    trade_count = 0   # Счетчик сделок
    line_count = 0  # Счетчик сделок по линейной регрессии
    #roll_window = 25  # Окно для линейной регрессии
    window = options[0]  # Окно для скользящего среднего и стандартного отклонения
    coef_entry = options[1]
    coef_sell = options[2]
    coef_loss = options[3]
    coef_profit = options[4]
    flag = True # Индикатор стоп-лосса
    long_position = False # Идикатор длинной позиции
    short_position = False # Идикатор короткой позиции

    # Объединение данных по дате
    z_test = pd.merge(ticker1, ticker2, left_index=True, right_index=True, how='inner')
    z_test.columns = ['JBHT_Close', 'LOW_Close']

    # Расчет логарифмической разности
    z_test['Spread'] = np.log(z_test['LOW_Close']) - np.log(z_test['JBHT_Close'])
    # Расчет z-score
    z_test['Mean_spread'] = z_test['Spread'].rolling(window=window).mean()
    z_test['Std_spread'] = z_test['Spread'].rolling(window=window).std()
    z_test['Z-score'] = (z_test['Spread'] - z_test['Mean_spread']) / z_test['Std_spread']
    #z_test['JBHT_mean'] = z_test['JBHT_Close'].rolling(window=10).mean()
    #z_test['LOW_mean'] = z_test['LOW_Close'].rolling(window=10).mean()
    #z_test['JBHT_std'] = z_test['JBHT_Close'].rolling(window=10).std()
    #z_test['LOW_std'] = z_test['LOW_Close'].rolling(window=10).std()

    for index in range(window, len(z_test)-1):

        current_date = z_test.index[index]
        open_date = z_test.index[index+1]
        jbht_close = z_test.at[current_date, 'JBHT_Close']
        low_close = z_test.at[current_date, 'LOW_Close']
        jbht_price = data_open.at[open_date, name_1]
        low_price = data_open.at[open_date, name_2]
        jbht_predict = line_reg.at[current_date, name_1]
        low_predict = line_reg.at[current_date, name_2]
        jbht_slope = slope.at[current_date, name_1]  # Коэффициент наклона
        low_slope = slope.at[current_date, name_2]  # Коэффициент наклона

        # Расчет скользящего среднего и стандартного отклонения для спреда
        std_spread = z_test.at[current_date, 'Std_spread']
        #mean_spread = z_test.at[current_date, 'Mean_spread']
        z_score = z_test.at[current_date, 'Z-score']

        # Отклонение цены акции от линии регрессии
        deviation_jbht = relative_deviation.at[current_date, name_1]
        deviation_low = relative_deviation.at[current_date, name_2]

        # Устанавливаем пороги на основе волатильности
        entry = coef_entry * std_spread
        sell = coef_sell * std_spread

        if flag == False:
            entry = 3 * coef_entry * std_spread
            sell = 3 * coef_sell * std_spread            
            if index > loss_date + 3:
                flag = True
                entry = coef_entry * std_spread
                sell = coef_sell * std_spread

        # Торговые сигналы и сделки
        # Покупка спреда
        #if flag:
        if z_score < -entry:
            # Длинная позиция по дорогой, короткая по дешевой
            if deviation_low < deviation_jbht:
                if low_shares == 0 and jbht_shares == 0:
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    cash -= low_shares * low_price
                    cash += -jbht_shares * jbht_price
                    trade_count += 2
                elif low_shares > 0 and jbht_shares < 0:
                    delta = (-cash / jbht_price - jbht_shares) * jbht_price
                    low_shares -= delta / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 1
                elif low_shares < 0 and jbht_shares > 0:
                    cash += jbht_shares * jbht_price
                    cash += low_shares * low_price
                    low_shares = 0
                    jbht_shares = 0
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 2
            # Короткая позиция по LOW, длинная по JBHT
            else: 
                if low_shares == 0 and jbht_shares == 0:
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    cash -= jbht_shares * jbht_price
                    cash += -low_shares * low_price
                    trade_count += 2
                elif low_shares < 0 and jbht_shares > 0:
                    delta = (-cash / low_price - low_shares) * low_price
                    jbht_shares -= delta / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 1
                elif low_shares > 0 and jbht_shares < 0:
                    cash += low_shares * low_price
                    cash += jbht_shares * jbht_price
                    low_shares = 0
                    jbht_shares = 0
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 2

        # Покупка спреда
        elif z_score > entry:
            # Короткая позиция по LOW, длинная по JBHT
            if deviation_jbht < deviation_low:
                if low_shares == 0 and jbht_shares == 0:
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    cash -= jbht_shares * jbht_price
                    cash -= low_shares * low_price
                    trade_count += 2
                elif low_shares < 0 and jbht_shares > 0:
                    delta = (-cash / low_price - low_shares) * low_price
                    jbht_shares -= delta / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 1
                elif low_shares > 0 and jbht_shares < 0:
                    cash += low_shares * low_price
                    cash += jbht_shares * jbht_price
                    low_shares = 0
                    jbht_shares = 0
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 2
            # Длинная позиция по LOW, короткая по JBHT
            else:
                if low_shares == 0 and jbht_shares == 0:
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    cash -= low_shares * low_price
                    cash += -jbht_shares * jbht_price
                    trade_count += 2
                elif low_shares > 0 and jbht_shares < 0:
                    delta = (-cash / jbht_price - jbht_shares) * jbht_price
                    low_shares -= delta / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 1
                elif low_shares < 0 and jbht_shares > 0:
                    cash += jbht_shares * jbht_price
                    cash += low_shares * low_price
                    low_shares = 0
                    jbht_shares = 0
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 2

        # Выход из позиции по спреду
        elif -sell < z_score < sell and low_shares != 0: #and line_position == False:
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 2
        
        # Покупка спреда при восходящей регрессии двух тикеров
        elif jbht_close > jbht_predict and low_close > low_predict and low_slope > 0.5 and jbht_slope > 0.5:
            if low_shares == 0 and jbht_shares == 0:
                low_shares = 0.5 * cash / low_price
                jbht_shares = 0.5 * cash / jbht_price
                cash -= low_shares * low_price
                cash -= jbht_shares * jbht_price
                line_count += 1
            elif low_shares < 0 and jbht_shares > 0:
                cash += low_shares * low_price
                low_shares = 0
                # jbht_shares -= 0.5 * jbht_shares
                # cash += jbht_shares * jbht_price
                # low_shares = cash / low_price
                # cash -= low_shares * low_price
                line_count += 1
                if cash >= 0:
                    jbht_shares += cash / jbht_price
                    cash = 0
                else:
                    jbht_shares -= cash / jbht_price
                    cash = 0
            elif low_shares > 0 and jbht_shares < 0:
                cash += jbht_shares * jbht_price
                jbht_shares = 0
                # low_shares -= 0.5 * low_shares
                # cash += low_shares * low_price
                # jbht_shares = cash / jbht_price
                # cash -= jbht_shares * jbht_price
                line_count += 1
                if cash >= 0:
                    low_shares += cash / low_price
                    cash = 0
                else:
                    low_shares -= cash / low_price
                    cash = 0
            long_position = True

        elif (jbht_close < jbht_predict and low_close < low_predict) and long_position == True: #or low_slope <= 0 and jbht_slope <= 0 and line_position == True:
            cash += jbht_shares * jbht_price
            cash += low_shares * low_price
            jbht_shares = 0
            low_shares = 0
            long_position = False
            line_count += 1
        
        # # Продажа спреда в шорт при нисходящей регрессии двух тикеров
        # elif jbht_close < jbht_predict and low_close < low_predict and low_slope < -0.4 and jbht_slope < -0.4:
        #     if low_shares == 0 and jbht_shares == 0:
        #         low_shares = -0.5 * cash / low_price
        #         jbht_shares = -0.5 * cash / jbht_price
        #         cash += low_shares * low_price
        #         cash += jbht_shares * jbht_price
        #         line_count += 1
        #     elif low_shares < 0 and jbht_shares > 0:
        #         jbht_shares = 0
        #         cash += jbht_shares * jbht_price
        #         #cash += 0.5 * low_shares * low_price
        #         line_count += 1
        #         # if cash >= 0:
        #         #     low_shares += cash / jbht_price
        #         #     cash = 0
        #         # else:
        #         #     jbht_shares -= cash / jbht_price
        #         #     cash = 0
        #     elif low_shares > 0 and jbht_shares < 0:
        #         cash += low_shares * low_price
        #         low_shares = 0
        #         line_count += 1
        #         # if cash >= 0:
        #         #     low_shares += cash / low_price
        #         #     cash = 0
        #         # else:
        #         #     low_shares -= cash / low_price
        #         #     cash = 0
        #     short_position = True

        # elif (jbht_close > jbht_predict and low_close > low_predict) and short_position == True: #or low_slope <= 0 and jbht_slope <= 0 and line_position == True:
        #     cash += jbht_shares * jbht_price
        #     cash += low_shares * low_price
        #     jbht_shares = 0
        #     low_shares = 0
        #     short_position = False
        #     line_count += 1

        # Установка стоп-лосса
        # Расчет текущей стоимости позиций         
        value_low = low_shares * low_close
        value_jbht = jbht_shares * jbht_close
        total_value = cash + value_low + value_jbht

        if max_cash < cash:
            max_cash = cash
        stop_loss = coef_loss * max_cash

        # Расчет текущей позиции
        current_loss = max_cash - total_value

        # Проверка условий стоп-лосса
        if current_loss > stop_loss:
            # Закрытие всех позиций
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 1
            max_cash = cash
            flag = False
            loss_date = index
                
        # Установка take-profit
        stop_profit = coef_profit * max_cash

        # Расчет текущего профита
        current_profit = total_value - max_cash

        if current_profit > stop_profit:
            # Закрытие всех позиций
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 1

        # Если есть шорт позиции, списываем комиссию за день
        if jbht_shares < 0:
            daily_fee = -jbht_shares * jbht_price * 0.13 / 365
            cash -= daily_fee
        elif low_shares < 0:
            daily_fee = -low_shares * low_price * 0.13 / 365
            cash -= daily_fee
            
            #z_test.at[current_date, 'Cash'] = cash
                   
    return {'pair': pair, 'final_result': cash, 'Position' : total_value, 'trade_count': trade_count, 'line_count' : line_count}

In [10]:
def pairs_trading_heavy(pair, options):
    # Тестируемые тикеры акций
    name_1, name_2 = pair

    ticker1 = data[name_1]
    ticker2 = data[name_2]

    # Начальные параметры
    initial_capital = 100000    # Стартовый капитал
    max_cash = initial_capital    # Максимально достигнутая сумма
    cash = initial_capital    # Баланс кошелька
    low_shares = 0    # Акции low
    jbht_shares = 0    # Акции jbht
    trade_count = 0   # Счетчик сделок
    line_count = 0  # Счетчик сделок по линейной регрессии
    window = options[0]  # Окно для скользящего среднего и стандартного отклонения
    window_regression = options[1]
    coef_long = options[2]
    roll_window = options[3]  # Окно для линейной регрессии
    coef_entry = options[4]
    coef_sell = options[5]
    coef_loss = options[6]
    coef_profit = options[7]
    #coef_short = options[8]

    flag = True # Индикатор стоп-лосса
    long_position = False # Идикатор длинной позиции
    short_position = False # Идикатор короткой позиции

    # Объединение данных по дате
    z_test = pd.merge(ticker1, ticker2, left_index=True, right_index=True, how='inner')
    z_test.columns = ['JBHT_Close', 'LOW_Close']

    # Расчет логарифмической разности
    z_test['Spread'] = np.log(z_test['LOW_Close']) - np.log(z_test['JBHT_Close'])
    # Расчет z-score
    z_test['Mean_spread'] = z_test['Spread'].rolling(window=window).mean()
    z_test['Std_spread'] = z_test['Spread'].rolling(window=window).std()
    z_test['Z-score'] = (z_test['Spread'] - z_test['Mean_spread']) / z_test['Std_spread']
    z_test['DateNumeric'] = z_test.index.map(pd.Timestamp.toordinal)        # Перевод дат в цифровой формат
    X = z_test['DateNumeric'].values.reshape(-1, 1)     # Создание индексов в качестве признаков
    #z_test['JBHT_mean'] = z_test['JBHT_Close'].rolling(window=10).mean()
    #z_test['LOW_mean'] = z_test['LOW_Close'].rolling(window=10).mean()
    #z_test['JBHT_std'] = z_test['JBHT_Close'].rolling(window=10).std()
    #z_test['LOW_std'] = z_test['LOW_Close'].rolling(window=10).std()

    for index in range(window, len(z_test)-1):

        current_date = z_test.index[index]
        numeric_date = current_date.toordinal()
        open_date = z_test.index[index+1]
        jbht_close = z_test.at[current_date, 'JBHT_Close']
        low_close = z_test.at[current_date, 'LOW_Close']
        jbht_price = data_open.at[open_date, name_1]
        low_price = data_open.at[open_date, name_2]
        #jbht_predict = line_reg.at[current_date, name_1]
        #low_predict = line_reg.at[current_date, name_2]
        #jbht_slope = slope.at[current_date, name_1]  # Коэффициент наклона
        #low_slope = slope.at[current_date, name_2]  # Коэффициент наклона

        # Расчет скользящего среднего и стандартного отклонения для спреда
        # Модель для спреда
        # model_spread = LinearRegression()
        # model_spread.fit(X[index-window:index+1], z_test['Spread'][index-window:index+1])
        # spread_predict = model_spread.predict([[numeric_date]])

        std_spread = z_test.at[current_date, 'Std_spread']
        #mean_spread = z_test.at[current_date, 'Mean_spread']
        z_score = z_test.at[current_date, 'Z-score']
        
        # Отклонение цены акции от линии регрессии для Z-score
        # Модель для JBHT
        model_jbht = LinearRegression()
        model_jbht.fit(X[index-roll_window:index+1], z_test['JBHT_Close'][index-roll_window:index+1])
        predicted_jbht = model_jbht.predict([[numeric_date]])
        # Модель для LOW
        model_low = LinearRegression()
        model_low.fit(X[index-roll_window:index+1], z_test['LOW_Close'][index-roll_window:index+1])
        predicted_low = model_low.predict([[numeric_date]])
        # Расчет отклонения от линии регрессии
        deviation_jbht = (jbht_close - predicted_jbht) / predicted_jbht
        deviation_low = (low_close - predicted_low) / predicted_low

        # Линейная регрессия для длинной и короткой позиций
        # Модель для JBHT
        model_jbht_long = LinearRegression()
        model_jbht_long.fit(X[index-window_regression:index+1], z_test['JBHT_Close'][index-window_regression:index+1])
        predicted_jbht_long = model_jbht_long.predict([[numeric_date]])
        # Модель для LOW
        model_low_long = LinearRegression()
        model_low_long.fit(X[index-window_regression:index+1], z_test['LOW_Close'][index-window_regression:index+1])
        predicted_low_long = model_low_long.predict([[numeric_date]])
        # Расчет отклонения от линии регрессии
        #deviation_jbht_long = (jbht_price - predicted_jbht_long) / predicted_jbht_long
        #deviation_low_long = (low_price - predicted_low_long) / predicted_low_long
        jbht_slope = model_jbht_long.coef_[0]  # Коэффициент наклона
        low_slope = model_jbht_long.coef_[0]  # Коэффициент наклона

        # Устанавливаем пороги на основе волатильности
        entry = coef_entry * std_spread
        sell = coef_sell * std_spread

        if flag == False:
            entry = 3 * coef_entry * std_spread
            sell = 3 * coef_sell * std_spread            
            if index > loss_date + 2:
                flag = True
                entry = coef_entry * std_spread
                sell = coef_sell * std_spread

        # Торговые сигналы и сделки
        # Покупка спреда
        #if flag:
        # Покупка спреда при восходящей регрессии двух тикеров
        if jbht_close > predicted_jbht_long and low_close > predicted_low_long and low_slope > coef_long and jbht_slope > coef_long:
            if low_shares == 0 and jbht_shares == 0:
                low_shares = 0.5 * cash / low_price
                jbht_shares = 0.5 * cash / jbht_price
                cash -= low_shares * low_price
                cash -= jbht_shares * jbht_price
                line_count += 1
            elif low_shares < 0 and jbht_shares > 0:
                cash += low_shares * low_price
                low_shares = 0
                line_count += 1
                jbht_shares += cash / jbht_price
                cash = 0
            elif low_shares > 0 and jbht_shares < 0:
                cash += jbht_shares * jbht_price
                jbht_shares = 0
                line_count += 1
                low_shares += cash / low_price
                cash = 0
            long_position = True

        elif (jbht_close < predicted_jbht_long or low_close < predicted_low_long) and long_position == True:
            cash += jbht_shares * jbht_price
            cash += low_shares * low_price
            jbht_shares = 0
            low_shares = 0
            long_position = False
            line_count += 1

        # Стратегия z-счет
        elif z_score < -entry and short_position == False and long_position == False:
            # Длинная позиция по дорогой, короткая по дешевой
            if deviation_low < deviation_jbht:
                if low_shares == 0 and jbht_shares == 0:
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    cash -= low_shares * low_price
                    cash += -jbht_shares * jbht_price
                    trade_count += 2
                elif low_shares > 0 and jbht_shares < 0:
                    delta = (-cash / jbht_price - jbht_shares) * jbht_price
                    low_shares -= delta / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 1
                elif low_shares < 0 and jbht_shares > 0:
                    cash += jbht_shares * jbht_price
                    cash += low_shares * low_price
                    low_shares = 0
                    jbht_shares = 0
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 2
            # Короткая позиция по LOW, длинная по JBHT
            else: 
                if low_shares == 0 and jbht_shares == 0:
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    cash -= jbht_shares * jbht_price
                    cash += -low_shares * low_price
                    trade_count += 2
                elif low_shares < 0 and jbht_shares > 0:
                    delta = (-cash / low_price - low_shares) * low_price
                    jbht_shares -= delta / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 1
                elif low_shares > 0 and jbht_shares < 0:
                    cash += low_shares * low_price
                    cash += jbht_shares * jbht_price
                    low_shares = 0
                    jbht_shares = 0
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 2

        # Покупка спреда
        elif z_score > entry and short_position == False and long_position == False:
            # Короткая позиция по LOW, длинная по JBHT
            if deviation_jbht < deviation_low:
                if low_shares == 0 and jbht_shares == 0:
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    cash -= jbht_shares * jbht_price
                    cash -= low_shares * low_price
                    trade_count += 2
                elif low_shares < 0 and jbht_shares > 0:
                    delta = (-cash / low_price - low_shares) * low_price
                    jbht_shares -= delta / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 1
                elif low_shares > 0 and jbht_shares < 0:
                    cash += low_shares * low_price
                    cash += jbht_shares * jbht_price
                    low_shares = 0
                    jbht_shares = 0
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 2
            # Длинная позиция по LOW, короткая по JBHT
            else:
                if low_shares == 0 and jbht_shares == 0:
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    cash -= low_shares * low_price
                    cash += -jbht_shares * jbht_price
                    trade_count += 2
                elif low_shares > 0 and jbht_shares < 0:
                    delta = (-cash / jbht_price - jbht_shares) * jbht_price
                    low_shares -= delta / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 1
                elif low_shares < 0 and jbht_shares > 0:
                    cash += jbht_shares * jbht_price
                    cash += low_shares * low_price
                    low_shares = 0
                    jbht_shares = 0
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 2

        # Выход из позиции по спреду
        elif -sell < z_score < sell and low_shares != 0 and long_position == False and short_position == False:
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 2

        # #Продажа спреда в шорт при нисходящей регрессии двух тикеров
        # elif jbht_close < predicted_jbht_long and low_close < predicted_low_long and low_slope < -coef_short and jbht_slope < -coef_short:
        #     if low_shares == 0 and jbht_shares == 0:
        #         low_shares = (-0.5 * cash) / low_price
        #         jbht_shares = (-0.5 * cash) / jbht_price
        #         cash -= low_shares * low_price
        #         cash -= jbht_shares * jbht_price
        #         line_count += 1
        #     elif low_shares < 0 and jbht_shares > 0:
        #         cash += jbht_shares * jbht_price
        #         jbht_shares = 0
        #         line_count += 1
        #     elif low_shares > 0 and jbht_shares < 0:
        #         cash += low_shares * low_price
        #         low_shares = 0
        #         line_count += 1
        #     short_position = True

        # elif (jbht_close > predicted_jbht_long or low_close > predicted_low_long) and short_position == True: #or low_slope <= 0 and jbht_slope <= 0 and line_position == True:
        #     cash += jbht_shares * jbht_price
        #     cash += low_shares * low_price
        #     jbht_shares = 0
        #     low_shares = 0
        #     short_position = False
        #     line_count += 1

        # Установка стоп-лосса
        # Расчет текущей стоимости позиций         
        value_low = low_shares * low_close
        value_jbht = jbht_shares * jbht_close
        total_value = cash + value_low + value_jbht

        if max_cash < cash:
            max_cash = cash
        stop_loss = coef_loss * max_cash

        # Расчет текущей позиции
        current_loss = max_cash - total_value

        # Проверка условий стоп-лосса
        if current_loss > stop_loss:
            # Закрытие всех позиций
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 1
            max_cash = cash
            flag = False
            loss_date = index
                
        # Установка take-profit
        stop_profit = coef_profit * max_cash

        # Расчет текущего профита
        current_profit = total_value - max_cash

        if current_profit > stop_profit:
            # Закрытие всех позиций
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 1

        # Если есть шорт позиции, списываем комиссию за день
        if jbht_shares < 0:
            daily_fee = -jbht_shares * jbht_price * 0.13 / 365
            cash -= daily_fee
        elif low_shares < 0:
            daily_fee = -low_shares * low_price * 0.13 / 365
            cash -= daily_fee
            
            #z_test.at[current_date, 'Cash'] = cash
                   
    return {'pair': pair, 'final_result': cash, 'Position' : total_value, 'trade_count': trade_count, 'line_count' : line_count}

In [1268]:
# Оптимальные параметры:  [24, 9, 35, 1.0, 0.09, 0.1, 0])
window = 30
window_regression = 9
coef_long = 0.5
coef_short = 1
roll_window = 23
coef_entry = 33
coef_sell = 1.38
coef_loss = 0.09
coef_profit = 0.14
options = [window, window_regression, coef_long, coef_short, roll_window, coef_entry, coef_sell, coef_loss, coef_profit]
pairs_trading_heavy(('ETN', 'AMAT'), options)

{'pair': ('ETN', 'AMAT'),
 'final_result': 218555.3525344053,
 'Position': 207204.7865482352,
 'trade_count': 225,
 'line_count': 47}

In [1271]:
def parameters(pair):
    # Стартовые параметры
    # Оптимальные параметры:  [20.12715593  8.65949107  0.14615061 12.60124853 31.07073092  1.38127755, 0.15555868  0.14070436]
    window = 30
    window_regression = 9
    coef_long = 0.35
    roll_window = 21
    coef_entry = 35
    coef_sell = 1
    coef_loss = 0.08
    coef_profit = 0.1
    options = [window, window_regression, coef_long, roll_window, coef_entry, coef_sell, coef_loss, coef_profit, coef_short]
    # Определите диапазоны для каждого параметра
    window_range = np.arange(20, 36, 1)
    roll_window_range = np.arange(7, 28, 1)
    window_regression_range = np.arange(5, 15, 1)
    coef_long_range = (0, 0.6, 0.05)
    coef_entry_range = np.arange(25, 40, 1)
    coef_sell_range = np.arange(1, 4, 0.5)
    coef_loss_range = np.arange(0.02, 0.16, 0.01)
    coef_profit_range = np.arange(0.06, 0.2, 0.02)
    best_score = 0
    
    for i in window_range:
        for j in roll_window_range:
            if i >= j:
            # Вызываем функцию торговли с текущими параметрами
                options[0] = i
                options[3] = j
                score = pairs_trading_heavy(pair, options)['Position']

                # Сохраняем параметры, если текущий результат лучше предыдущего
                if score >= best_score:
                    best_score = score
                    window = i
                    roll_window = j

    options[0] = window
    options[3] = roll_window

    for i in window_regression_range:
        for j in coef_long_range:
            if i <= window:
                options[1] = i
                options[2] = j
                # Вызываем функцию торговли с текущими параметрами
                score = pairs_trading_heavy(pair, options)['Position']
                # Сохраняем параметры, если текущий результат лучше предыдущего
                if score >= best_score:
                    best_score = score
                    window_regression = i
                    coef_long = j
                    
    options[1] = window_regression
    options[2] = coef_long
    
    for i in coef_entry_range:
        for j in coef_sell_range:
            options[4] = i 
            options[5] = j
            # Вызываем функцию торговли с текущими параметрами
            score = pairs_trading_heavy(pair, options)['Position']

            # Сохраняем параметры, если текущий результат лучше предыдущего
            if score >= best_score:
                best_score = score
                coef_entry = i
                coef_sell = j

    options[4] = coef_entry
    options[5] = coef_sell

    for i in coef_loss_range:
        for j in coef_profit_range:
            options[6] = i
            options[7] = j
            # Вызываем функцию торговли с текущими параметрами
            iteraciya = pairs_trading_heavy(pair, options)
            score = iteraciya['Position']

            # Сохраняем параметры, если текущий результат лучше предыдущего
            if score >= best_score:
                best_score = score
                coef_loss = i
                coef_profit = j
                best_iter = iteraciya
    
    options[6] = coef_loss
    options[7] = coef_profit

    return best_iter, options

In [1078]:
parameters(('ANET', 'CDNS'))

({'pair': ('ANET', 'CDNS'),
  'final_result': 678341.3619222196,
  'Position': 678341.3619222196,
  'trade_count': 264,
  'line_count': 58},
 [22, 13, 30, 1.0, 0.10999999999999999, 0.2, 0.35, 9])

In [15]:
window = 22  # Окно для скользящего среднего и стандартного отклонения
window_regression = 13
coef_long = 0.35
roll_window = 9  # Окно для линейной регрессии
coef_entry = 30
coef_sell = 1
coef_loss = 0.11
coef_profit = 0.2
options = [window, window_regression, coef_long, roll_window, coef_entry, coef_sell, coef_loss, coef_profit]
pairs_trading_heavy(('ANET', 'CDNS'), options)

{'pair': ('ANET', 'CDNS'),
 'final_result': 223699.7348152561,
 'Position': 223699.7348152561,
 'trade_count': 280,
 'line_count': 52}

In [410]:
def pre_parametrs(pair):
    # Стартовые параметры
    window = 28
    coef_entry = 35
    coef_sell = 1.5
    coef_loss = 0.09
    coef_profit = 0.11
    options = [window, coef_entry, coef_sell, coef_loss, coef_profit]
    # Определите диапазоны для каждого параметра
    window_range = np.arange(25, 34, 1)
    coef_entry_range = np.arange(30, 40, 1)
    coef_sell_range = np.arange(1, 4, 0.5)
    coef_loss_range = np.arange(0.01, 0.16, 0.01)
    coef_profit_range = np.arange(0.05, 0.18, 0.01)
    best_score = 0
    best_params = {}
    results = []

    for i in window_range:
        # Вызываем функцию торговли с текущими параметрами
        options[0] = i
        score = pairs_trading(pair, options)['Position']

        # Сохраняем параметры, если текущий результат лучше предыдущего
        if score > best_score:
            best_score = score
            window = i

    options[0] = window

    for i in coef_entry_range:
        # Вызываем функцию торговли с текущими параметрами
        options[1] = i
        score = pairs_trading(pair, options)['Position']

        # Сохраняем параметры, если текущий результат лучше предыдущего
        if score > best_score:
            best_score = score
            coef_entry = i

    options[1] = coef_entry

    return {'pair' : pair, 'cash' : best_score, 'prams' : options}

In [ ]:
def roi(window, roll_window, coef_entry, coef_sell, window_regression, coef_long, coef_loss, coef_profit):
    options = []
    options.append(window, roll_window, coef_entry, coef_sell, window_regression, coef_long, coef_loss, coef_profit)
    return pairs_trading(pair, options)

In [1081]:
def objective_function(x):
    window, window_regression, coef_long, roll_window, coef_entry, coef_sell, coef_loss, coef_profit = x

    # Проверка условий
    if roll_window > window or window_regression > window:
        return float('inf')  # Возвращаем бесконечность, если условие нарушено

    options = [round(window), round(window_regression), round(coef_long, 3), round(roll_window), round(coef_entry, 2), round(coef_sell, 2), round(coef_loss, 3), round(coef_profit, 3)]
    pair = ('ANET', 'CDNS')
    result = pairs_trading_heavy(pair, options)  
    score = result['Position']  # Пример использования возвращаемого значения
    # PSO минимизирует функцию, поэтому, если нужно максимизировать score, возвращаем -score
    return -score

# Определение границ параметров
lb = [18, 5, 0, 7, 25, 0.5, 0.02, 0.05]  # Нижние границы
ub = [33, 15, 0.7, 25, 50, 7, 0.2, 0.25]  # Верхние границы

# Запуск PSO
xopt, fopt = pso(objective_function, lb, ub, swarmsize=150, maxiter=300)

# Вывод результатов
print("Оптимальные параметры: ", xopt)
print("Лучшая метрика score: ", -fopt)

# Stopping search: maximum iterations reached --> 200
# Оптимальные параметры:  [27.63475659 12.10472817 29.32283894  1.          5.20531653  0.1
#   0.13518865  0.17      ]
# Лучшая метрика score:  1002340.4374007218

Stopping search: maximum iterations reached --> 300
Оптимальные параметры:  [26.81796056  8.84014138  0.34825466 14.92788798 28.59553532  3.38718879
  0.10286727  0.14792464]
Лучшая метрика score:  626007.2195037592


In [1082]:
import requests

def send_telegram_message(message):
    # Параметры для отправки сообщения
    telegram_token = '6737079836:AAHQT4Bgz3Kh8LRA_kaQDJ76HJBzh9S_X8A'
    chat_id = '264028245'
    url = f'https://api.telegram.org/bot{token}/sendMessage'
    payload = {
        'chat_id': chat_id,
        'text': message,
        'parse_mode': 'Markdown'
    }
    response = requests.post(url, json=payload)
    return response.json()

message = f"Оптимальные параметры: {xopt}\nЛучшая метрика score: {-fopt}"
# Отправка сообщения
send_telegram_message(message)

{'ok': True,
 'result': {'message_id': 10,
  'from': {'id': 6737079836,
   'is_bot': True,
   'first_name': 'MyResults',
   'username': 'resultsofmystrategy_bot'},
  'chat': {'id': 264028245,
   'first_name': 'Иван',
   'last_name': 'Сухов',
   'username': 'sukhov_is',
   'type': 'private'},
  'date': 1707204823,
  'text': 'Оптимальные параметры: 26.81796056  8.84014138  0.34825466 14.92788798 28.59553532  3.38718879\n  0.10286727  0.14792464\nЛучшая метрика score: 626007.2195037592'}}

In [1083]:
def objective_function(x):
    window, window_regression, coef_long, roll_window, coef_entry, coef_sell, coef_loss, coef_profit = x

    # Проверка условий
    if roll_window > window or window_regression > window:
        return float('inf')  # Возвращаем бесконечность, если условие нарушено

    options = [round(window), round(window_regression), round(coef_long, 3), round(roll_window), round(coef_entry, 2), round(coef_sell, 2), round(coef_loss, 3), round(coef_profit, 3)]
    pair = ('ANET', 'CDNS')
    result = pairs_trading_heavy(pair, options)  
    score = result['Position']  # Пример использования возвращаемого значения
    # PSO минимизирует функцию, поэтому, если нужно максимизировать score, возвращаем -score
    return -score

# Определение границ параметров
lb = [18, 5, 0, 7, 25, 0.5, 0.02, 0.05]  # Нижние границы
ub = [33, 15, 0.7, 25, 50, 7, 0.2, 0.25]  # Верхние границы

# Запуск PSO
xopt, fopt = pso(objective_function, lb, ub, swarmsize=100, maxiter=200)

# Вывод результатов
print("Оптимальные параметры: ", xopt)
print("Лучшая метрика score: ", -fopt)

Stopping search: maximum iterations reached --> 200
Оптимальные параметры:  [20.12715593  8.65949107  0.14615061 12.60124853 31.07073092  1.38127755
  0.15555868  0.14070436]
Лучшая метрика score:  741211.3848874782


In [ ]:
results = []
results_2 = []
for pair in pairs:
    results.append(pairs_trading(pair, options))
for pair in results[:100]['Pair']:
    results_2.append(pre_parametrs(pair))
sort results_2
for pair in results_2[:5]:
    results_3.append(parametrs((pair, options)))
sort results_3
for pair in results_3[:3]:
    final.append(parametrs_roi(results_3[0], options))
sort final
pairs_trading_heavy(final[0], options, data)

In [1084]:
message = f"Оптимальные параметры: {xopt}\nЛучшая метрика score: {-fopt}"
send_telegram_message(telegram_token, chat_id, message)

{'ok': True,
 'result': {'message_id': 11,
  'from': {'id': 6737079836,
   'is_bot': True,
   'first_name': 'MyResults',
   'username': 'resultsofmystrategy_bot'},
  'chat': {'id': 264028245,
   'first_name': 'Иван',
   'last_name': 'Сухов',
   'username': 'sukhov_is',
   'type': 'private'},
  'date': 1707216066,
  'text': 'Оптимальные параметры: 20.12715593  8.65949107  0.14615061 12.60124853 31.07073092  1.38127755\n  0.15555868  0.14070436\nЛучшая метрика score: 741211.3848874782'}}

In [929]:
parameters(('ANET', 'CDNS'))

({'pair': ('ANET', 'CDNS'),
  'final_result': 559215.0003402415,
  'Position': 559215.0003402415,
  'trade_count': 178,
  'line_count': 94},
 [32, 17, 25, 3.5, 0.15999999999999998, 0.08, 0])

In [1272]:
for pair in [('JBHT', 'LOW'), ('MAR', 'PSX'), ('ETN', 'AMAT'), ('ELV', 'NOC'), ('ANET', 'CDNS')]:
    print(parameters(pair))

({'pair': ('JBHT', 'LOW'), 'final_result': 358415.9921064687, 'Position': 361333.6038240416, 'trade_count': 229, 'line_count': 40}, [30, 9, 0.35, 18, 35, 2.5, 0.039999999999999994, 0.16000000000000003, 1])
({'pair': ('MAR', 'PSX'), 'final_result': 0.0, 'Position': 629984.6840574059, 'trade_count': 196, 'line_count': 33}, [23, 12, 0.6, 21, 34, 1.0, 0.039999999999999994, 0.2, 1])
({'pair': ('ETN', 'AMAT'), 'final_result': 442529.40135192417, 'Position': 419546.8520223458, 'trade_count': 264, 'line_count': 30}, [28, 10, 0.6, 22, 30, 1.0, 0.15999999999999998, 0.2, 1])
({'pair': ('ELV', 'NOC'), 'final_result': 555512.2185918374, 'Position': 575070.8757720173, 'trade_count': 248, 'line_count': 55}, [32, 9, 0.35, 21, 36, 3.5, 0.06999999999999999, 0.12000000000000001, 1])
({'pair': ('ANET', 'CDNS'), 'final_result': 678341.3619222196, 'Position': 678341.3619222196, 'trade_count': 264, 'line_count': 58}, [22, 9, 0.35, 13, 30, 1.0, 0.10999999999999999, 0.2, 1])


In [ ]:
pairs_trading(('JBHT', 'LOW'), )

In [286]:
options = [28, 35, 2.5, 0.06, 0.06]
pairs_trading(('JBHT', 'LOW'), options)

{'pair': ('JBHT', 'LOW'),
 'final_result': 215159.9659887338,
 'Position': 212930.37354248774,
 'trade_count': 294,
 'line_count': 0}

[28, 35, 1.5, 0.1, 0.8]

In [497]:
# Тест быстродействия
test_pair = filtred_pairs[:100]
result_test = []
options = [28, 35, 1.5, 0.1, 0.15]
start_time = time.time()
for pair in test_pair:
    result_test.append(pairs_trading(pair, options))
end_time = time.time()
print(end_time - start_time)

2.167736053466797


In [527]:
pre_results = []
start_time = time.time()
for pair in filtred_pairs[:100]:
    pre_results.append(pre_parametrs(pair))
pre_df = pd.DataFrame(pre_results)
pre_df = pre_df.sort_values(by='cash', ascending=False)
end_time = time.time()
print(end_time - start_time)

40.48433589935303


In [528]:
pre_df[:20]

,pair,cash,prams
56,"(AAPL, CRM)",383255.545513,"[28, 34, 1.5, 0.09, 0.11]"
54,"(AAPL, CMG)",328020.148466,"[29, 36, 1.5, 0.09, 0.11]"
99,"(AAPL, TYL)",292744.177814,"[26, 32, 1.5, 0.09, 0.11]"
12,"(AAL, BBY)",268676.679500,"[25, 36, 1.5, 0.09, 0.11]"
29,"(AAL, PYPL)",254856.489189,"[25, 34, 1.5, 0.09, 0.11]"
75,"(AAPL, MAR)",240015.676300,"[25, 30, 1.5, 0.09, 0.11]"
3,"(A, HBAN)",237209.175229,"[28, 36, 1.5, 0.09, 0.11]"
47,"(AAPL, BKNG)",228762.895094,"[31, 37, 1.5, 0.09, 0.11]"
86,"(AAPL, NXPI)",221100.855624,"[25, 30, 1.5, 0.09, 0.11]"
85,"(AAPL, NXPI)",221100.855624,"[25, 30, 1.5, 0.09, 0.11]"


In [ ]:
result_test[:30]

In [525]:
# Топорный тест всех пар на стратегии
list_results = []
options = [28, 36, 1.5, 0.12, 0.15]
for pair in pairs:
    #if pair[0] in data and pair[1] in data:
    list_results.append(pairs_trading(pair, options))
# Создание DataFrame из списка словарей
df_4 = pd.DataFrame(list_results)
df_4 = df_4.sort_values(by='Position', ascending=False)
df_4[:20]

,pair,final_result,Position,trade_count,line_count
95706,"(JNPR, PODD)",629005.945587,621525.609224,150,0
114620,"(PAYC, PSA)",577688.163815,576756.688897,135,11
16684,"(ANSS, MPWR)",545771.343933,545771.343933,156,17
48069,"(COO, UHS)",523055.747338,520287.118817,247,8
19486,"(APTV, ODFL)",503252.322266,518791.128845,172,10
42757,"(CI, MRO)",540883.681768,510556.294983,138,0
36857,"(CB, FANG)",495130.185175,477846.660680,211,7
34082,"(BX, PAYC)",489231.444977,476435.241053,175,8
114635,"(PAYC, RMD)",460578.934845,463425.464101,164,10
16099,"(ANET, FSLR)",464455.745189,452474.220794,118,4


In [145]:
df_4[:50]

,pair,final_result,Position,trade_count,line_count
12713,"(CTLT, FE)",448513.729296,545083.600506,134,0
9186,"(CDAY, META)",461916.830133,523426.264517,156,0
12744,"(CTLT, SBAC)",372752.382611,446598.718514,138,0
5081,"(AXON, META)",401684.484595,427063.181796,121,0
25777,"(META, MPC)",412979.322096,424921.832809,96,0
9051,"(CCL, LULU)",404681.491786,406107.931027,93,0
12702,"(CTLT, DTE)",333400.615535,394003.035007,126,0
28590,"(PKG, RCL)",384749.842692,384749.842692,110,0
9022,"(CCL, FDX)",373176.286231,373176.286231,86,0
5435,"(BA, BIIB)",327451.514216,355261.363619,173,0


In [35]:
df_4[:50]

,pair,final_result,trade_count
43161,"(CI, MRO)",1.187244e+06,107
114635,"(PAYC, RMD)",6.839870e+05,145
31471,"(BMY, MRO)",6.610768e+05,117
109200,"(MTCH, ON)",6.553169e+05,73
53635,"(CTRA, KDP)",6.480532e+05,114
4219,"(ADBE, KHC)",6.178207e+05,108
78480,"(FTNT, LVS)",5.863414e+05,90
80608,"(GL, MPWR)",5.784719e+05,66
63111,"(DXCM, LVS)",5.641363e+05,80
25003,"(BALL, TFX)",5.560041e+05,139


In [9]:
df_3.to_csv('SP500 pairs - 42-1std-28d.csv')

In [ ]:
# Использование ThreadPoolExecutor для многопоточной обработки
result_test = []
start_time = time.time()
with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    # Запуск функции process_pair на каждой паре в списке pairs
    futures = [executor.submit(process_pair, pair) for pair in pairs[:10]]

    for future in concurrent.futures.as_completed(futures):
        result_test.append(future.result())
end_time = time.time()
print(end_time - start_time)

In [7]:
def test_function(pair):
    name_1, name_2 = pair
    name_tick1 = str(name_1)
    name_tick2 = str(name_2)
    return (name_tick1, name_tick2)

In [ ]:
result_test = []
start_time = time.time()
# Использование ThreadPoolExecutor для многопоточной обработки
with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    # Запуск функции process_pair на каждой паре в списке pairs
    results = executor.map(test_function, pairs[:10])

    for i in results:
        result_test.append(i)
end_time = time.time()
print(end_time - start_time)

In [20]:
test_function(test_pair[1])

('A', 'AAPL')

In [8]:
def chunkify(lst, n):
    """ Разделение списка на n подсписков """
    return [lst[i::n] for i in range(n)]

def worker(data, queue):
    print(f"Обработка данных: {data}")  # Логирование
    for item in data:
        result = test_function(item)
        queue.put(result)
    print(f"Обработка завершена для данных: {data}")  # Логирование

if __name__ == '__main__':
    start_time = time.time()

    num_processes = 4  # Количество процессов
    
    # Предположим, что test_pair - это список кортежей
    data_chunks = chunkify(test_pair, num_processes)
    queue = multiprocessing.Queue()

    processes = [multiprocessing.Process(target=worker, args=(chunk, queue)) for chunk in data_chunks]

    for p in processes:
        p.start()

    for p in processes:
        p.join()

    results = []
    while not queue.empty():
        results.append(queue.get())

    end_time = time.time()
    print(end_time - start_time)
    print(results)  # Вывод результатов

0.06100106239318848
[]


In [17]:
def test_function(pair):
    name_1, name_2 = pair
    name_tick1 = str(name_1)
    name_tick2 = str(name_2)
    return (name_tick1, name_tick2)

def chunkify(lst, n):
    """ Разделение списка на n подсписков """
    return [lst[i::n] for i in range(n)]

def worker(data, queue):
    print(f"Запуск worker с данными: {data}")
    for item in data:
        result = test_function(item)
        queue.put(result)
    print(f"Worker завершил работу с данными: {data}")

if __name__ == '__main__':
    start_time = time.time()

    num_processes = 4

    data_chunks = chunkify(test_pair, num_processes)
    queue = multiprocessing.Queue()

    processes = [multiprocessing.Process(target=worker, args=(chunk, queue)) for chunk in data_chunks]

    for p in processes:
        p.start()

    for p in processes:
        p.join()

    results = []
    while not queue.empty():
        results.append(queue.get())

    end_time = time.time()
    print(f"Время выполнения: {end_time - start_time}")
    print(f"Результаты: {results}")

In [27]:
df_4[:50]

,pair,final_result,trade_count
6128,"(ADSK, HIG)",522201.853612,110
80859,"(GLW, KMX)",502133.035664,111
6227,"(ADSK, MS)",488337.769731,128
61061,"(DPZ, PAYC)",472146.281299,95
62405,"(DVA, LRCX)",461186.777178,44
78214,"(FSLR, MTD)",454119.290031,69
114685,"(PAYC, TSN)",448021.973770,77
24990,"(BALL, SWK)",443817.121161,131
90325,"(IFF, KLAC)",440039.016828,94
25817,"(BBWI, PAYC)",435921.939925,57


In [8]:
list_results = []
# Топорный тест всех пар на стратегии
for pair in pairs:
    list_results.append(process_pair(pair))
# Создание DataFrame из списка словарей
df_3 = pd.DataFrame(list_results)
df_3 = df_3.sort_values(by='final_result', ascending=False)
df_3.head()

,pair,final_result,trade_count
114635,"(PAYC, RMD)",734113.626899,121
77548,"(FOXA, KMX)",651511.734828,111
25821,"(BBWI, PEAK)",562155.870128,105
12317,"(AMAT, ETN)",552845.606184,147
76215,"(FIS, WHR)",521283.629097,123


In [58]:
df_3k = df_4[:10000].copy()

In [59]:
df_3k['ADF'] = 0
df_3k['ADF-stat'] = 0.0
df_3k['ADF-p'] = 0.0
df_3k['CADF-score'] = 0.0
df_3k['CADF-p'] = 0.0
df_3k['Spearman'] = 0.0
df_3k['spear-p'] = 0.0

df_3k.head()

,pair,final_result,Position,trade_count,ADF,ADF-stat,ADF-p,CADF-score,CADF-p,Spearman,spear-p
15991,"(ANET, CDNS)",0.000000,738675.472279,132,0,0.0,0.0,0.0,0.0,0.0,0.0
119544,"(ROK, TSLA)",661665.012013,661665.012013,101,0,0.0,0.0,0.0,0.0,0.0,0.0
16099,"(ANET, FSLR)",0.000000,550197.875981,93,0,0.0,0.0,0.0,0.0,0.0,0.0
67647,"(EOG, ON)",0.000000,547088.228080,90,0,0.0,0.0,0.0,0.0,0.0,0.0
78219,"(FSLR, NEE)",535938.098453,535938.098453,84,0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
# Тесты пар
for i, row in df_3k.iterrows():
    tik1, tik2 = row['pair']
    if tik1 in data.columns and tik2 in data.columns:
        stock1 = data[tik1]
        stock2 = data[tik2]

        # Рассчитываем коинтеграцию с помощью CADF теста
        score, p_value, _ = ts.coint(stock1, stock2)
        # ADF тест
        spread = stock1 - stock2
        adf_result = ts.adfuller(spread)
        # Корреляция Спирмена
        corr, spear_p = spearmanr(stock1, stock2)
        # Сохраняем результаты
        df_3k.at[i, 'CADF-p'] = p_value
        df_3k.at[i, 'CADF-score'] = score
        df_3k.at[i, 'ADF-p'] = adf_result[1]
        df_3k.at[i, 'ADF-stat'] = adf_result[0]
        if adf_result[0] < adf_result[4]['1%']:
            df_3k.at[i, 'ADF'] = 1
        df_3k.at[i, 'Spearman'] = corr
        df_3k.at[i, 'spear-p'] = spear_p

df_3k = df_3k.sort_values(by='Position')
df_3k[-20:]

,pair,final_result,Position,trade_count,ADF,ADF-stat,ADF-p,CADF-score,CADF-p,Spearman,spear-p
107522,"(MPC, WDC)",508531.404350,475262.837499,90,0,-2.068887,0.257208,-0.606807,0.955799,-0.218480,6.784407e-07
112851,"(NXPI, PSX)",0.000000,477571.914093,75,1,-3.535959,0.007110,-3.458963,0.036211,0.469049,4.254446e-29
48460,"(COO, UHS)",485295.378180,480659.158707,141,0,-2.698407,0.074314,-3.080789,0.092227,0.732500,2.232191e-86
99242,"(LDOS, STLD)",0.000000,486433.548294,130,0,-1.952300,0.307871,-2.023168,0.516516,0.146415,9.447636e-04
24119,"(BA, TSLA)",487098.561610,487098.561610,72,0,-1.801693,0.379621,-1.805070,0.627256,-0.116509,8.642828e-03
58904,"(DHR, PAYC)",476021.098099,488134.138735,127,0,-1.682926,0.439930,-3.297919,0.055021,0.641335,4.344424e-60
54303,"(CTVA, FANG)",499913.525551,490850.783274,145,0,-1.939690,0.313626,-2.032981,0.511397,-0.077258,8.223057e-02
35836,"(CAH, STLD)",0.000000,496210.437825,91,0,-2.827329,0.054475,-2.132694,0.459294,0.777085,1.305324e-103
38889,"(CDAY, DD)",500635.318740,500635.318740,124,1,-4.070836,0.001082,-4.439532,0.001537,0.798018,3.754894e-113
45838,"(CMG, FSLR)",0.000000,503333.313691,96,0,-0.605040,0.869839,-0.869011,0.925011,0.630185,1.799578e-57


In [41]:
df_3k = df_3k.sort_values(by='final_result', ascending=False)
df_3k[:50]

,pair,final_result,trade_count,ADF,ADF-stat,ADF-p,CADF-score,CADF-p,Spearman,spear-p
43161,"(CI, MRO)",1.187244e+06,107,0,-2.322018,0.164965,-2.674443,0.208905,0.554549,3.208336e-42
114635,"(PAYC, RMD)",6.839870e+05,145,0,-2.640941,0.084827,-2.781261,0.171720,0.752879,8.505842e-94
31471,"(BMY, MRO)",6.610768e+05,117,0,-0.777606,0.825612,-0.537749,0.961533,0.278369,1.782598e-10
109200,"(MTCH, ON)",6.553169e+05,73,0,-2.519773,0.110737,-2.919363,0.130638,-0.601493,3.326621e-51
53635,"(CTRA, KDP)",6.480532e+05,114,0,-3.113618,0.025562,-3.355010,0.047599,0.206632,2.706657e-06
4219,"(ADBE, KHC)",6.178207e+05,108,0,-0.645727,0.860311,-1.380498,0.804120,-0.493147,1.986459e-32
78480,"(FTNT, LVS)",5.863414e+05,90,0,-2.465095,0.124200,-3.040504,0.100889,0.428725,4.386229e-24
80608,"(GL, MPWR)",5.784719e+05,66,0,-2.059917,0.260935,-1.733227,0.661553,0.300926,4.511414e-12
63111,"(DXCM, LVS)",5.641363e+05,80,0,-2.397947,0.142317,-2.471277,0.291881,0.653411,4.764533e-63
25003,"(BALL, TFX)",5.560041e+05,139,0,-2.393785,0.143498,-3.663754,0.020386,0.784292,8.937571e-107


In [14]:
st = sum(df_3k['final_result']/len(df_3k))
st

146929.81905414883

In [57]:
st = sum(df_4[:]['final_result'])/len(df_4)
st

74274.95774487982

In [1]:
process_pair(('MAR', 'PSX'))
plt.figure(figsize=(12, 6))

plt.subplot(2, 1, 1)
plt.plot(z_test.index, z_test['JBHT_Close'], label=f'{name_1}')
plt.plot(z_test.index, z_test['LOW_Close'], label=f'{name_2}')
plt.title(f'Тест пары {name_1}&{name_2}')
plt.xlabel('Date')
plt.ylabel('Dynamic')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(z_test.index, z_test['Cash'], label='Cash')
plt.xlabel('Date')
plt.ylabel('Cash')
plt.legend()

plt.show()

NameError: name 'process_pair' is not defined

In [ ]:
def pairs_trading_heavy(pair, options): #window, coef_entry, coef_sell, coef_loss, coef_profit):
    # Тестируемые тикеры акций
    name_1, name_2 = pair

    ticker1 = data[name_1]
    ticker2 = data[name_2]

    # Начальные параметры
    initial_capital = 100000    # Стартовый капитал
    max_cash = initial_capital    # Максимально достигнутая сумма
    cash = initial_capital    # Баланс кошелька
    low_shares = 0    # Акции low
    jbht_shares = 0    # Акции jbht
    trade_count = 0   # Счетчик сделок
    line_count = 0  # Счетчик сделок по линейной регрессии
    window = options[0]  # Окно для скользящего среднего и стандартного отклонения
    roll_window = options[1]  # Окно для линейной регрессии
    coef_entry = options[2]
    coef_sell = options[3]
    window_regression = options[4]
    coef_long = options[5]
    coef_loss = options[6]
    coef_profit = options[7]
    flag = True # Индикатор стоп-лосса
    long_position = False # Идикатор длинной позиции
    short_position = False # Идикатор короткой позиции

    # Объединение данных по дате
    z_test = pd.merge(ticker1, ticker2, left_index=True, right_index=True, how='inner')
    z_test.columns = ['JBHT_Close', 'LOW_Close']

    # Расчет логарифмической разности
    z_test['Spread'] = np.log(z_test['LOW_Close']) - np.log(z_test['JBHT_Close'])
    # Расчет z-score
    z_test['Mean_spread'] = z_test['Spread'].rolling(window=window).mean()
    z_test['Std_spread'] = z_test['Spread'].rolling(window=window).std()
    z_test['Z-score'] = (z_test['Spread'] - z_test['Mean_spread']) / z_test['Std_spread']
    z_test['DateNumeric'] = z_test.index.map(pd.Timestamp.toordinal)        # Перевод дат в цифровой формат
    X = z_test['DateNumeric'].values.reshape(-1, 1)     # Создание индексов в качестве признаков
    #z_test['JBHT_mean'] = z_test['JBHT_Close'].rolling(window=10).mean()
    #z_test['LOW_mean'] = z_test['LOW_Close'].rolling(window=10).mean()
    #z_test['JBHT_std'] = z_test['JBHT_Close'].rolling(window=10).std()
    #z_test['LOW_std'] = z_test['LOW_Close'].rolling(window=10).std()

    for index in range(window, len(z_test)-1):

        current_date = z_test.index[index]
        open_date = z_test.index[index+1]
        jbht_close = z_test.at[current_date, 'JBHT_Close']
        low_close = z_test.at[current_date, 'LOW_Close']
        jbht_price = data_open.at[open_date, name_1]
        low_price = data_open.at[open_date, name_2]
        jbht_predict = line_reg.at[current_date, name_1]
        low_predict = line_reg.at[current_date, name_2]
        #jbht_slope = slope.at[current_date, name_1]  # Коэффициент наклона
        #low_slope = slope.at[current_date, name_2]  # Коэффициент наклона

        # Расчет скользящего среднего и стандартного отклонения для спреда
        std_spread = z_test.at[current_date, 'Std_spread']
        #mean_spread = z_test.at[current_date, 'Mean_spread']
        z_score = z_test.at[current_date, 'Z-score']

        # Линейная регрессия для парной торговли
        numeric_date = current_date.toordinal()
        # Модель для JBHT
        model_jbht = LinearRegression()
        model_jbht.fit(X[index-roll_window:index+1], z_test['JBHT_Close'][index-roll_window:index+1])
        predicted_jbht = model_jbht.predict([[numeric_date]])
        # Модель для LOW
        model_low = LinearRegression()
        model_low.fit(X[index-roll_window:index+1], z_test['LOW_Close'][index-roll_window:index+1])
        predicted_low = model_low.predict([[numeric_date]])
        # Расчет отклонения от линии регрессии
        deviation_jbht = (jbht_price - predicted_jbht) / predicted_jbht
        deviation_low = (low_price - predicted_low) / predicted_low

        # Линейная регрессия для длинной и короткой позиций
        numeric_date = current_date.toordinal()
        # Модель для JBHT
        model_jbht_long = LinearRegression()
        model_jbht_long.fit(X[index-window_regression:index+1], z_test['JBHT_Close'][index-window_regression:index+1])
        predicted_jbht_long = model_jbht_long.predict([[numeric_date]])
        # Модель для LOW
        model_low_long = LinearRegression()
        model_low_long.fit(X[index-window_regression:index+1], z_test['LOW_Close'][index-window_regression:index+1])
        predicted_low_long = model_low_long.predict([[numeric_date]])
        # Расчет отклонения от линии регрессии
        deviation_jbht_long = (jbht_price - predicted_jbht) / predicted_jbht
        deviation_low_long = (low_price - predicted_low) / predicted_low
        jbht_slope = model_jbht_long.coef_[0]  # Коэффициент наклона
        low_slope = model_jbht_long.coef_[0]  # Коэффициент наклона

        # Устанавливаем пороги на основе волатильности
        entry = coef_entry * std_spread
        sell = coef_sell * std_spread

        if flag == False:
            entry = 3 * coef_entry * std_spread
            sell = 3 * coef_sell * std_spread            
            if index > loss_date + 3:
                flag = True
                entry = coef_entry * std_spread
                sell = coef_sell * std_spread

        # Торговые сигналы и сделки
        # Покупка спреда
        #if flag:
        if z_score < -entry:
            # Длинная позиция по дорогой, короткая по дешевой
            if deviation_low < deviation_jbht:
                if low_shares == 0 and jbht_shares == 0:
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    cash -= low_shares * low_price
                    cash += -jbht_shares * jbht_price
                    trade_count += 2
                elif low_shares > 0 and jbht_shares < 0:
                    delta = (-cash / jbht_price - jbht_shares) * jbht_price
                    low_shares -= delta / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 1
                elif low_shares < 0 and jbht_shares > 0:
                    cash += jbht_shares * jbht_price
                    cash += low_shares * low_price
                    low_shares = 0
                    jbht_shares = 0
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 2
            # Короткая позиция по LOW, длинная по JBHT
            else: 
                if low_shares == 0 and jbht_shares == 0:
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    cash -= jbht_shares * jbht_price
                    cash += -low_shares * low_price
                    trade_count += 2
                elif low_shares < 0 and jbht_shares > 0:
                    delta = (-cash / low_price - low_shares) * low_price
                    jbht_shares -= delta / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 1
                elif low_shares > 0 and jbht_shares < 0:
                    cash += low_shares * low_price
                    cash += jbht_shares * jbht_price
                    low_shares = 0
                    jbht_shares = 0
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 2

        # Покупка спреда
        elif z_score > entry:
            # Короткая позиция по LOW, длинная по JBHT
            if deviation_jbht < deviation_low:
                if low_shares == 0 and jbht_shares == 0:
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    cash -= jbht_shares * jbht_price
                    cash -= low_shares * low_price
                    trade_count += 2
                elif low_shares < 0 and jbht_shares > 0:
                    delta = (-cash / low_price - low_shares) * low_price
                    jbht_shares -= delta / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 1
                elif low_shares > 0 and jbht_shares < 0:
                    cash += low_shares * low_price
                    cash += jbht_shares * jbht_price
                    low_shares = 0
                    jbht_shares = 0
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 2
            # Длинная позиция по LOW, короткая по JBHT
            else:
                if low_shares == 0 and jbht_shares == 0:
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    cash -= low_shares * low_price
                    cash += -jbht_shares * jbht_price
                    trade_count += 2
                elif low_shares > 0 and jbht_shares < 0:
                    delta = (-cash / jbht_price - jbht_shares) * jbht_price
                    low_shares -= delta / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 1
                elif low_shares < 0 and jbht_shares > 0:
                    cash += jbht_shares * jbht_price
                    cash += low_shares * low_price
                    low_shares = 0
                    jbht_shares = 0
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 2

        # Выход из позиции по спреду
        elif -sell < z_score < sell and low_shares != 0: #and line_position == False:
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 2
        
        # Покупка спреда при восходящей регрессии двух тикеров
        elif jbht_close > jbht_predict and low_close > low_predict and low_slope > 0.5 and jbht_slope > 0.5:
            if low_shares == 0 and jbht_shares == 0:
                low_shares = 0.5 * cash / low_price
                jbht_shares = 0.5 * cash / jbht_price
                cash -= low_shares * low_price
                cash -= jbht_shares * jbht_price
                line_count += 1
            elif low_shares < 0 and jbht_shares > 0:
                cash += low_shares * low_price
                low_shares = 0
                # jbht_shares -= 0.5 * jbht_shares
                # cash += jbht_shares * jbht_price
                # low_shares = cash / low_price
                # cash -= low_shares * low_price
                line_count += 1
                if cash >= 0:
                    jbht_shares += cash / jbht_price
                    cash = 0
                else:
                    jbht_shares -= cash / jbht_price
                    cash = 0
            elif low_shares > 0 and jbht_shares < 0:
                cash += jbht_shares * jbht_price
                jbht_shares = 0
                # low_shares -= 0.5 * low_shares
                # cash += low_shares * low_price
                # jbht_shares = cash / jbht_price
                # cash -= jbht_shares * jbht_price
                line_count += 1
                if cash >= 0:
                    low_shares += cash / low_price
                    cash = 0
                else:
                    low_shares -= cash / low_price
                    cash = 0
            long_position = True

        elif (jbht_close < jbht_predict and low_close < low_predict) and long_position == True: #or low_slope <= 0 and jbht_slope <= 0 and line_position == True:
            cash += jbht_shares * jbht_price
            cash += low_shares * low_price
            jbht_shares = 0
            low_shares = 0
            long_position = False
            line_count += 1
        
        # # Продажа спреда в шорт при нисходящей регрессии двух тикеров
        # elif jbht_close < jbht_predict and low_close < low_predict and low_slope < -0.4 and jbht_slope < -0.4:
        #     if low_shares == 0 and jbht_shares == 0:
        #         low_shares = -0.5 * cash / low_price
        #         jbht_shares = -0.5 * cash / jbht_price
        #         cash += low_shares * low_price
        #         cash += jbht_shares * jbht_price
        #         line_count += 1
        #     elif low_shares < 0 and jbht_shares > 0:
        #         jbht_shares = 0
        #         cash += jbht_shares * jbht_price
        #         #cash += 0.5 * low_shares * low_price
        #         line_count += 1
        #         # if cash >= 0:
        #         #     low_shares += cash / jbht_price
        #         #     cash = 0
        #         # else:
        #         #     jbht_shares -= cash / jbht_price
        #         #     cash = 0
        #     elif low_shares > 0 and jbht_shares < 0:
        #         cash += low_shares * low_price
        #         low_shares = 0
        #         line_count += 1
        #         # if cash >= 0:
        #         #     low_shares += cash / low_price
        #         #     cash = 0
        #         # else:
        #         #     low_shares -= cash / low_price
        #         #     cash = 0
        #     short_position = True

        # elif (jbht_close > jbht_predict and low_close > low_predict) and short_position == True: #or low_slope <= 0 and jbht_slope <= 0 and line_position == True:
        #     cash += jbht_shares * jbht_price
        #     cash += low_shares * low_price
        #     jbht_shares = 0
        #     low_shares = 0
        #     short_position = False
        #     line_count += 1

        # Установка стоп-лосса
        # Расчет текущей стоимости позиций         
        value_low = low_shares * low_close
        value_jbht = jbht_shares * jbht_close
        total_value = cash + value_low + value_jbht

        if max_cash < cash:
            max_cash = cash
        stop_loss = coef_loss * max_cash

        # Расчет текущей позиции
        current_loss = max_cash - total_value

        # Проверка условий стоп-лосса
        if current_loss > stop_loss:
            # Закрытие всех позиций
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 1
            max_cash = cash
            flag = False
            loss_date = index
                
        # Установка take-profit
        stop_profit = coef_profit * max_cash

        # Расчет текущего профита
        current_profit = total_value - max_cash

        if current_profit > stop_profit:
            # Закрытие всех позиций
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 1

        # Если есть шорт позиции, списываем комиссию за день
        if jbht_shares < 0:
            daily_fee = -jbht_shares * jbht_price * 0.13 / 365
            cash -= daily_fee
        elif low_shares < 0:
            daily_fee = -low_shares * low_price * 0.13 / 365
            cash -= daily_fee
            
            #z_test.at[current_date, 'Cash'] = cash
                   
    return {'pair': pair, 'final_result': cash, 'Position' : total_value, 'trade_count': trade_count, 'line_count' : line_count}

In [ ]:
def parameters(pair):
    # Стартовые параметры
    window = 28
    roll_window = 23
    coef_entry = 35
    coef_sell = 1.5
    window_regression = 10
    coef_long = 0.5
    coef_loss = 0.15
    coef_profit = 0.17
    options = [window, roll_window, coef_entry, coef_sell, window_regression, coef_long, coef_loss, coef_profit]
    # Определите диапазоны для каждого параметра
    window_range = np.arange(20, 36, 1)
    roll_window_range = np.arange(7, 36, 1)
    coef_entry_range = np.arange(25, 40, 1)
    coef_sell_range = np.arange(1, 4, 0.5)
    window_regression_range = np.arange(3, 25, 1)
    coef_long_range = np.arange(0.1, 0.7, 0.1)
    coef_loss_range = np.arange(0.02, 0.16, 0.01)
    coef_profit_range = np.arange(0.06, 0.2, 0.02)
    best_score = 0
    best_params = {}
    results = []
    
    for i in window_range:
        for j in roll_window_range:
            if i >= j:
            # Вызываем функцию торговли с текущими параметрами
                options[0] = i
                options[1] = j
                score = pairs_trading_heavy(pair, options)['Position']
                # Сохраняем параметры, если текущий результат лучше предыдущего
                if score >= best_score:
                    best_score = score
                    window = i
                    roll_window = j

    options[0] = window
    options[1] = roll_window

    for i in coef_entry_range:
        for j in coef_sell_range:
            options[2] = i 
            options[3] = j
            # Вызываем функцию торговли с текущими параметрами
            score = pairs_trading_heavy(pair, options)['Position']
            # Сохраняем параметры, если текущий результат лучше предыдущего
            if score >= best_score:
                best_score = score
                coef_entry = i
                coef_sell = j

    options[2] = coef_entry
    options[3] = coef_sell

    for i in window_regression_range:
        for j in coef_long_range:
            if i <= window:
                options[4] = i
                options[5] = j
                # Вызываем функцию торговли с текущими параметрами
                score = pairs_trading_heavy(pair, options)['Position']
                # Сохраняем параметры, если текущий результат лучше предыдущего
                if score >= best_score:
                    best_score = score
                    window_regression = i
                    coef_long = j

    options[4] = window_regression
    options[5] = coef_long
                
    for i in coef_loss_range:
        for j in coef_profit_range:
            options[6] = i
            options[7] = j
            # Вызываем функцию торговли с текущими параметрами
            iteraciya = pairs_trading_heavy(pair, options)
            #if isinstance(iteraciya['Position'], float) and iteraciya['Position'] > 0:
            score = iteraciya['Position']
            # Сохраняем параметры, если текущий результат лучше предыдущего
            if score >= best_score:
                best_score = score
                coef_loss = i
                coef_profit = j
                best_iter = iteraciya
    
    options[6] = coef_loss
    options[7] = coef_profit

    return best_iter, options